* Denna Notebook [26_aikstats.ipynb](https://github.com/salgo60/ifkdb/blob/main/Notebook/26_aikstats.ipynb)
* [#26](https://github.com/salgo60/ifkdb/issues/26)

* AIK
   * sv:Wikipedia [Kategori:Fotbollsspelare_i_AIK](https://sv.wikipedia.org/wiki/Kategori:Fotbollsspelare_i_AIK)
   * https://www.aikstats.se/player.php
   * Wikidata SPARQL https://w.wiki/J3Rk
   * FB https://www.facebook.com/AIKStatistikdatabas/

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2026-03-01 08:29:40.941654


In [2]:
import requests
import urllib3

print("requests:", requests.__version__)
print("urllib3:", urllib3.__version__)

requests: 2.32.5
urllib3: 2.6.3


In [3]:
# If needed:
# !pip install requests beautifulsoup4 pandas lxml mwclient tqdm rapidfuzz

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from rapidfuzz import fuzz, process
import mwclient

In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "https://www.aikstats.se/player.php?who=all"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (research bot; contact: salgo60@msn.com)"
}

response = requests.get(BASE_URL, headers=HEADERS, timeout=20)
response.raise_for_status()

soup = BeautifulSoup(response.text, "lxml")

players = []

for row in soup.find_all("tr", class_="normal"):
    cols = row.find_all("td")
    if len(cols) < 4:
        continue

    link = cols[0].find("a", href=True)
    if not link:
        continue

    name = link.get_text(strip=True)
    href = link["href"]

    try:
        pid = int(href.split("id=")[1])
    except:
        continue

    position = cols[1].get_text(strip=True)
    years = cols[2].get_text(strip=True)
    matches = cols[3].get_text(strip=True)

    players.append({
        "id": pid,
        "name": name,
        "position": position,
        "years_active": years,
        "matches": matches
    })

df = pd.DataFrame(players)

print(f"Hittade {len(df)} spelare")
df.head()

Hittade 1307 spelare


,id,name,position,years_active,matches
0,4,"Per ""Pertan"" Karlsson",Back,1998-2022,531
1,8,"Daniel ""Tjerna"" Tjernström",Mittfältare,1999-2013,481
2,200,"Nils-Eric ""Nisse"" Johansson",Back,1997-2018,450
3,543,"Sven ""Dala"" Dahlkvist",Innerback,1974-1987,356
4,270,"Gustav ""Gurra"" Sjöberg",Målvakt,1929-1950,347


In [8]:
df_aik = pd.DataFrame(players)
df_aik.head()

,id,name,position,years_active,matches
0,4,"Per ""Pertan"" Karlsson",Back,1998-2022,531
1,8,"Daniel ""Tjerna"" Tjernström",Mittfältare,1999-2013,481
2,200,"Nils-Eric ""Nisse"" Johansson",Back,1997-2018,450
3,543,"Sven ""Dala"" Dahlkvist",Innerback,1974-1987,356
4,270,"Gustav ""Gurra"" Sjöberg",Målvakt,1929-1950,347


In [9]:
def chunked(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i + size] 
        
import mwclient
import pandas as pd

site = mwclient.Site("sv.wikipedia.org")

category = site.categories["Fotbollsspelare i AIK"]

rows = []

pages = [p for p in category.members() if p.namespace == 0]
titles = [p.name for p in pages]

for batch in chunked(titles, 50):
    result = site.api(
        action="query",
        titles="|".join(batch),
        prop="pageprops",
        ppprop="wikibase_item",
        format="json"
    )

    for page in result["query"]["pages"].values():
        rows.append({
            "wiki_name": page.get("title"),
            "wikidata_id": page.get("pageprops", {}).get("wikibase_item")
        })

df_wiki = pd.DataFrame(rows)

print(f"Hittade {len(df_wiki)} Wikipedia-spelare")
df_wiki.head()

Hittade 443 Wikipedia-spelare


,wiki_name,wikidata_id
0,Abdihakin Ali,Q119509691
1,Amin Affane,Q795483
2,Ammar Ahmed,Q19724740
3,Anders Almgren (fotbollsspelare),Q182613
4,Andreas Alm,Q495742


In [51]:
import requests
import pandas as pd

SPARQL = """
SELECT ?player ?playerLabel ?birthDate WHERE {
  ?player wdt:P54 wd:Q221602 .
  OPTIONAL { ?player wdt:P569 ?birthDate. }
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "sv,en,mul".
  }
}
"""

session = requests.Session()
session.headers.update({
    "User-Agent": "AIKStats research script (salgo60@msn.com)",
    "Accept": "application/sparql-results+json"
})

url = "https://query.wikidata.org/sparql"

response = session.get(
    url,
    params={"query": SPARQL},
    timeout=30
)

response.raise_for_status()
data = response.json()

wikidata_players = []

for r in data["results"]["bindings"]:
    wikidata_players.append({
        "wikidata_uri": r["player"]["value"],
        "name": r["playerLabel"]["value"],
        "birthDate": r.get("birthDate", {}).get("value")
    })

df_wd = pd.DataFrame(wikidata_players)
df_wd["wikidata_id"] = df_wd["wikidata_uri"].str.extract(r"(Q\d+)")
df_wd["wikidata_url"] = "https://www.wikidata.org/wiki/" + df_wd["wikidata_id"]
print(f"Wikidata P54 AIK: {len(df_wd)}")
df_wd.head()

Wikidata P54 AIK: 548


,wikidata_uri,name,birthDate,wikidata_id,wikidata_url
0,http://www.wikidata.org/entity/Q10792,Joel Ekstrand,1989-02-04T00:00:00Z,Q10792,https://www.wikidata.org/wiki/Q10792
1,http://www.wikidata.org/entity/Q74152,Viktor Fischer,1994-06-09T00:00:00Z,Q74152,https://www.wikidata.org/wiki/Q74152
2,http://www.wikidata.org/entity/Q81838,Kennedy Igboananike,1989-02-26T00:00:00Z,Q81838,https://www.wikidata.org/wiki/Q81838
3,http://www.wikidata.org/entity/Q81842,Robin Quaison,1993-10-09T00:00:00Z,Q81842,https://www.wikidata.org/wiki/Q81842
4,http://www.wikidata.org/entity/Q83638,Sebastian Larsson,1985-06-06T00:00:00Z,Q83638,https://www.wikidata.org/wiki/Q83638


In [52]:
# Wikipedia katergorier   
df_wiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   wiki_name    443 non-null    object
 1   wikidata_id  443 non-null    object
dtypes: object(2)
memory usage: 7.1+ KB


In [53]:

print(f"AIKStats spelare: {len(df_aik )}")
df_aik.head()

AIKStats spelare: 1307


,id,name,position,years_active,matches,norm_name,lastname
0,4,"Per ""Pertan"" Karlsson",Back,1998-2022,531,per karlsson,karlsson
1,8,"Daniel ""Tjerna"" Tjernström",Mittfältare,1999-2013,481,daniel tjernstrom,tjernstrom
2,200,"Nils-Eric ""Nisse"" Johansson",Back,1997-2018,450,nils eric johansson,johansson
3,543,"Sven ""Dala"" Dahlkvist",Innerback,1974-1987,356,sven dahlkvist,dahlkvist
4,270,"Gustav ""Gurra"" Sjöberg",Målvakt,1929-1950,347,gustav sjoberg,sjoberg


In [54]:
df_aik.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1307 entries, 0 to 1306
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1307 non-null   int64 
 1   name          1307 non-null   object
 2   position      1307 non-null   object
 3   years_active  1307 non-null   object
 4   matches       1307 non-null   object
 5   norm_name     1307 non-null   object
 6   lastname      1307 non-null   object
dtypes: int64(1), object(6)
memory usage: 71.6+ KB


In [55]:
wd_set = set(df_wd["wikidata_id"].dropna()) 
wiki_set = set(df_wiki["wikidata_id"].dropna())
wd_not_wiki = wd_set - wiki_set
print("WD men ej Wikipedia:", len(wd_not_wiki))

WD men ej Wikipedia: 105


In [56]:
# Wikipedia men ej WD: 
df_wd[df_wd["wikidata_id"].isin(wd_not_wiki)]

,wikidata_uri,name,birthDate,wikidata_id,wikidata_url
42,http://www.wikidata.org/entity/Q377362,Mike Kjølø,1971-10-27T00:00:00Z,Q377362,https://www.wikidata.org/wiki/Q377362
48,http://www.wikidata.org/entity/Q432898,Lisa De Vanna,1984-11-14T00:00:00Z,Q432898,https://www.wikidata.org/wiki/Q432898
61,http://www.wikidata.org/entity/Q527119,Emelie Ölander,1989-06-23T00:00:00Z,Q527119,https://www.wikidata.org/wiki/Q527119
66,http://www.wikidata.org/entity/Q582164,Nik Mrdja,1978-11-30T00:00:00Z,Q582164,https://www.wikidata.org/wiki/Q582164
75,http://www.wikidata.org/entity/Q711644,Arthur Svensson,1916-06-06T00:00:00Z,Q711644,https://www.wikidata.org/wiki/Q711644
...,...,...,...,...,...
535,http://www.wikidata.org/entity/Q124623802,Justus Gustafsson,1894-09-25T00:00:00Z,Q124623802,https://www.wikidata.org/wiki/Q124623802
538,http://www.wikidata.org/entity/Q124694676,Einar Bohlin,1911-01-23T00:00:00Z,Q124694676,https://www.wikidata.org/wiki/Q124694676
539,http://www.wikidata.org/entity/Q124843140,Johny Murray,1960-09-25T00:00:00Z,Q124843140,https://www.wikidata.org/wiki/Q124843140
546,http://www.wikidata.org/entity/Q126453747,Göran Åslin,1937-10-24T00:00:00Z,Q126453747,https://www.wikidata.org/wiki/Q126453747


In [57]:
# jämför mot AIKStats 
import unicodedata
import re

def normalize(name):
    if not name:
        return None

    # Ta bort smeknamn i citationstecken
    name = re.sub(r'".*?"', '', name)

    # Unicode-normalisering
    name = unicodedata.normalize("NFKD", name)
    name = name.encode("ascii", "ignore").decode()

    # Ta bort skiljetecken
    name = re.sub(r"[^\w\s-]", "", name)

    # Normalisera bindestreck
    name = name.replace("-", " ")

    # Normalisera whitespace
    name = re.sub(r"\s+", " ", name)

    return name.lower().strip()

df_aik["norm_name"] = df_aik["name"].apply(normalize)
df_wd["norm_name"] = df_wd["name"].apply(normalize)

stats_set = set(df_aik["norm_name"].dropna())
wd_name_set = set(df_wd["norm_name"].dropna())

print("WD men ej AIKStats:", len(wd_name_set - stats_set))
print("AIKStats men ej WD:", len(stats_set - wd_name_set))

WD men ej AIKStats: 103
AIKStats men ej WD: 851


In [58]:
df_wd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   wikidata_uri  548 non-null    object
 1   name          548 non-null    object
 2   birthDate     547 non-null    object
 3   wikidata_id   548 non-null    object
 4   wikidata_url  548 non-null    object
 5   norm_name     548 non-null    object
dtypes: object(6)
memory usage: 25.8+ KB


In [44]:
df_aik["norm_name"] = df_aik["name"].apply(normalize)
df_wd["norm_name"] = df_wd["name"].apply(normalize)

df_wd["birth_year"] = pd.to_datetime(df_wd["birthDate"], errors="coerce").dt.year

In [46]:
exact_matches = df_aik.merge(
    df_wd,
    on="norm_name",
    how="inner",
    suffixes=("_aik", "_wd")
)

len_exact = len(exact_matches)
print("Exakta matchningar:", len_exact)

Exakta matchningar: 455


In [47]:
matched_names = set(exact_matches["norm_name"])

aik_unmatched = df_aik[~df_aik["norm_name"].isin(matched_names)]
wd_unmatched = df_wd[~df_wd["norm_name"].isin(matched_names)]

print("AIKStats utan WD:", len(aik_unmatched))
print("WD utan AIKStats:", len(wd_unmatched))

AIKStats utan WD: 854
WD utan AIKStats: 103


In [48]:
def lastname(name):
    if not name:
        return None
    return name.split()[-1]

df_aik["lastname"] = df_aik["norm_name"].apply(lastname)
df_wd["lastname"] = df_wd["norm_name"].apply(lastname)

lastname_matches = df_aik.merge(
    df_wd,
    on="lastname",
    how="inner"
)

In [50]:
total_aik = len(df_aik)
total_wd = df_wd["name"].nunique()
matched = len_exact
missing_in_wd = total_aik - matched
wd_extra = total_wd - matched

report = {
    "AIKStats total IDs": total_aik,
    "Wikidata AIK Fotboll objects": total_wd,
    "Exact matches": matched,
    "AIKStats missing in WD": missing_in_wd,
    "WD objects not in AIKStats": wd_extra,
    "Coverage %": round((matched / total_aik) * 100, 2)
}

report

{'AIKStats total IDs': 1307,
 'Wikidata AIK Fotboll objects': 547,
 'Exact matches': 455,
 'AIKStats missing in WD': 852,
 'WD objects not in AIKStats': 92,
 'Coverage %': 34.81}

### Skapa final och rapport

In [59]:
df_final = df_aik.merge(
    df_wd,
    on="norm_name",
    how="inner",
    suffixes=("_aik", "_wd")
) 

print("Matchade rader:", len(df_final))
df_final.head()

Matchade rader: 455


,id,name_aik,position,years_active,matches,norm_name,lastname,wikidata_uri,name_wd,birthDate,wikidata_id,wikidata_url
0,4,"Per ""Pertan"" Karlsson",Back,1998-2022,531,per karlsson,karlsson,http://www.wikidata.org/entity/Q205479,Per Karlsson,1986-01-02T00:00:00Z,Q205479,https://www.wikidata.org/wiki/Q205479
1,8,"Daniel ""Tjerna"" Tjernström",Mittfältare,1999-2013,481,daniel tjernstrom,tjernstrom,http://www.wikidata.org/entity/Q1162941,Daniel Tjernström,1974-02-19T00:00:00Z,Q1162941,https://www.wikidata.org/wiki/Q1162941
2,200,"Nils-Eric ""Nisse"" Johansson",Back,1997-2018,450,nils eric johansson,johansson,http://www.wikidata.org/entity/Q699824,Nils-Eric Johansson,1980-01-13T00:00:00Z,Q699824,https://www.wikidata.org/wiki/Q699824
3,543,"Sven ""Dala"" Dahlkvist",Innerback,1974-1987,356,sven dahlkvist,dahlkvist,http://www.wikidata.org/entity/Q1368236,Sven Dahlkvist,1955-05-30T00:00:00Z,Q1368236,https://www.wikidata.org/wiki/Q1368236
4,270,"Gustav ""Gurra"" Sjöberg",Målvakt,1929-1950,347,gustav sjoberg,sjoberg,http://www.wikidata.org/entity/Q1369574,Gustav Sjöberg,1913-03-23T00:00:00Z,Q1369574,https://www.wikidata.org/wiki/Q1369574


In [60]:
df_merged = df_aik.merge(
    df_wd,
    on="norm_name",
    how="outer",
    indicator=True,
    suffixes=("_aik", "_wd")
) 
print("Rader:", len(df_merged))
df_merged.head()

Rader: 1412


,id,name_aik,position,years_active,matches,norm_name,lastname,wikidata_uri,name_wd,birthDate,wikidata_id,wikidata_url,_merge
0,720.0,A Tilbom,,? - ?,1,a tilbom,tilbom,NaN,NaN,NaN,NaN,NaN,left_only
1,NaN,NaN,NaN,NaN,NaN,aaron stoch rydell,NaN,http://www.wikidata.org/entity/Q125506161,Aaron Stoch Rydell,2006-10-14T00:00:00Z,Q125506161,https://www.wikidata.org/wiki/Q125506161,right_only
2,1052.0,Abdi Yusuf,,? - ?,,abdi yusuf,yusuf,NaN,NaN,NaN,NaN,NaN,left_only
3,1313.0,Abdihakin Ali,Mittfältare,2012-2022,11,abdihakin ali,ali,http://www.wikidata.org/entity/Q119509691,Abdihakin Ali,2002-01-22T00:00:00Z,Q119509691,https://www.wikidata.org/wiki/Q119509691,both
4,1096.0,Abdou-Fatawou Dodja,Forward,? - ?,,abdou fatawou dodja,dodja,NaN,NaN,NaN,NaN,NaN,left_only


In [61]:
df_merged["_merge"].value_counts()

_merge
left_only     854
both          455
right_only    103
Name: count, dtype: int64

In [62]:
df_final = df_merged[df_merged["_merge"] == "both"].copy()

In [63]:
df_aik_only = df_merged[df_merged["_merge"] == "left_only"]
df_wd_only = df_merged[df_merged["_merge"] == "right_only"]

In [64]:
report = {
    "AIKStats total IDs": len(df_aik),
    "Wikidata AIK Fotboll objects": df_wd["wikidata_id"].nunique(),
    "Exact matches": len(df_final),
    "AIKStats missing in WD": len(df_aik_only),
    "WD objects not in AIKStats": len(df_wd_only),
    "Coverage %": round((len(df_final) / len(df_aik)) * 100, 2)
}

report

{'AIKStats total IDs': 1307,
 'Wikidata AIK Fotboll objects': 548,
 'Exact matches': 455,
 'AIKStats missing in WD': 854,
 'WD objects not in AIKStats': 103,
 'Coverage %': 34.81}

In [65]:
# Om > 0 → du har namn-kollisioner (vanligt med t.ex. "Bo Andersson").
df_merged["norm_name"].duplicated().sum()

14

In [66]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1412 entries, 0 to 1411
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   id            1309 non-null   float64 
 1   name_aik      1309 non-null   object  
 2   position      1309 non-null   object  
 3   years_active  1309 non-null   object  
 4   matches       1309 non-null   object  
 5   norm_name     1412 non-null   object  
 6   lastname      1309 non-null   object  
 7   wikidata_uri  558 non-null    object  
 8   name_wd       558 non-null    object  
 9   birthDate     557 non-null    object  
 10  wikidata_id   558 non-null    object  
 11  wikidata_url  558 non-null    object  
 12  _merge        1412 non-null   category
dtypes: category(1), float64(1), object(11)
memory usage: 134.0+ KB


In [67]:
import urllib.parse
import pandas as pd

def link_aik(id_value):
    if pd.isna(id_value):
        return ""
    id_int = int(id_value)
    return f"[{id_int}](https://www.aikstats.se/player.php?id={id_int})"

def link_wikidata(qid):
    if pd.isna(qid):
        return ""
    return f"[{qid}](https://www.wikidata.org/wiki/{qid})"

def link_svwiki(name):
    if pd.isna(name):
        return ""
    encoded = urllib.parse.quote(name.replace(" ", "_"))
    return f"[{name}](https://sv.wikipedia.org/wiki/{encoded})"

In [68]:
df_table = df_merged[df_merged["_merge"] == "both"].copy()

rows = []
rows.append("| Namn | AIK fotboll | Wikidata | Wikipedia |")
rows.append("|------|--------------|----------|------------|")

for _, row in df_table.iterrows():
    namn = row["name_aik"]
    aik_link = link_aik(row["id"])
    wd_link = link_wikidata(row["wikidata_id"])
    wiki_link = link_svwiki(row["name_wd"])

    rows.append(f"| {namn} | {aik_link} | {wd_link} | {wiki_link} |")

markdown_output = "\n".join(rows)

print(markdown_output)

| Namn | AIK fotboll | Wikidata | Wikipedia |
|------|--------------|----------|------------|
| Abdihakin Ali | [1313](https://www.aikstats.se/player.php?id=1313) | [Q119509691](https://www.wikidata.org/wiki/Q119509691) | [Abdihakin Ali](https://sv.wikipedia.org/wiki/Abdihakin_Ali) |
| Abdussalam Magashy | [1462](https://www.aikstats.se/player.php?id=1462) | [Q106463788](https://www.wikidata.org/wiki/Q106463788) | [Abdussalam Magashy](https://sv.wikipedia.org/wiki/Abdussalam_Magashy) |
| Adam Ben Lamin | [1220](https://www.aikstats.se/player.php?id=1220) | [Q74230146](https://www.wikidata.org/wiki/Q74230146) | [Adam Ben Lamin](https://sv.wikipedia.org/wiki/Adam_Ben_Lamin) |
| Admir Ćatović | [13](https://www.aikstats.se/player.php?id=13) | [Q919416](https://www.wikidata.org/wiki/Q919416) | [Admir Ćatović](https://sv.wikipedia.org/wiki/Admir_%C4%86atovi%C4%87) |
| Agustin Gomez | [1243](https://www.aikstats.se/player.php?id=1243) | [Q60733812](https://www.wikidata.org/wiki/Q60733812) | 

In [45]:
from datetime import datetime
end_time = datetime.now()

duration = end_time - start_time

print("\n===== Körningsrapport =====")
print("Starttid :", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Sluttid  :", end_time.strftime("%Y-%m-%d %H:%M:%S"))

total_seconds = int(duration.total_seconds())
hours = total_seconds // 3600
minutes = (total_seconds % 3600) // 60
seconds = total_seconds % 60

print(f"Körtid   : {hours}h {minutes}m {seconds}s")
print("===========================\n")


===== Körningsrapport =====
Starttid : 2026-03-01 08:29:40
Sluttid  : 2026-03-01 08:58:45
Körtid   : 0h 29m 4s

